In [ ]:
# import os
# import re
# import json
# import random
# from collections import defaultdict

# print("Bắt đầu quá trình tái cấu trúc dataset cedarbhsig-260...")

# # --- CẤU HÌNH ---
# # Đường dẫn gốc tới dataset bạn đã tìm thấy
# BASE_BHSIG_DIR = '/kaggle/input/cedarbhsig-260/'
# OUTPUT_FILE = '/kaggle/working/bhsig_restructured_split.json'
# NUM_BENGALI_TEST_USERS = 50  # Số người dùng Bengali để test
# NUM_HINDI_TEST_USERS = 30    # Số người dùng Hindi để test
# RANDOM_SEED = 42

# # --- CÁC THƯ MỤC NGUỒN ---
# # Lưu ý: Các đường dẫn này dựa trên mô tả của bạn. Hãy kiểm tra lại nếu cần.
# HINDI_GENUINE_DIR = os.path.join(BASE_BHSIG_DIR, 'Reconstructed_BHSig160-20250403T183033Z-001/Reconstructed_BHSig160/Genuine')
# HINDI_FORGED_DIR = os.path.join(BASE_BHSIG_DIR, 'Reconstructed_BHSig160-20250403T183033Z-001/Reconstructed_BHSig160/Forged')
# BENGALI_GENUINE_DIR = os.path.join(BASE_BHSIG_DIR, 'Reconstructed_BHSig100-20250403T193101Z-001/Reconstructed_BHSig100/Genuine')
# BENGALI_FORGED_DIR = os.path.join(BASE_BHSIG_DIR, 'Reconstructed_BHSig100-20250403T193101Z-001/Reconstructed_BHSig100/Forged')

# # --- HÀM TRÍCH XUẤT THÔNG TIN TỪ TÊN FILE ---
# def parse_filename(filename):
#     # Ví dụ: H-S-011-G-01.tif hoặc B-S-011-F-01.tif
#     parts = filename.split('-')
#     if len(parts) >= 4:
#         language = parts[0]  # 'H' hoặc 'B'
#         user_id = int(parts[2])
#         return language, user_id
#     return None, None

# # --- THU THẬP VÀ PHÂN LOẠI FILE ---
# all_files = defaultdict(lambda: {'genuine': [], 'forgery': []})
# source_dirs = {
#     'hindi_genuine': HINDI_GENUINE_DIR,
#     'hindi_forged': HINDI_FORGED_DIR,
#     'bengali_genuine': BENGALI_GENUINE_DIR,
#     'bengali_forged': BENGALI_FORGED_DIR
# }

# for key, directory in source_dirs.items():
#     if not os.path.exists(directory):
#         print(f"Cảnh báo: Không tìm thấy thư mục '{directory}'. Bỏ qua.")
#         continue
        
#     for filename in os.listdir(directory):
#         if filename.endswith('.tif'):
#             lang, user_id = parse_filename(filename)
#             if lang and user_id:
#                 full_path = os.path.join(directory, filename)
#                 # Tạo một ID người dùng duy nhất bằng cách kết hợp ngôn ngữ và ID
#                 unique_user_id = f"{lang}-{user_id}"
                
#                 if 'genuine' in key:
#                     all_files[unique_user_id]['genuine'].append(full_path)
#                 elif 'forged' in key:
#                     all_files[unique_user_id]['forgery'].append(full_path)

# print(f"Đã xử lý và phân loại file cho {len(all_files)} người dùng.")

# # --- CHỌN NGẪU NHIÊN NGƯỜI DÙNG ĐỂ TEST ---
# bengali_users = [uid for uid in all_files if uid.startswith('B-')]
# hindi_users = [uid for uid in all_files if uid.startswith('H-')]

# random.seed(RANDOM_SEED)
# random.shuffle(bengali_users)
# random.shuffle(hindi_users)

# test_bengali_ids = bengali_users[:NUM_BENGALI_TEST_USERS]
# test_hindi_ids = hindi_users[:NUM_HINDI_TEST_USERS]

# # --- TẠO FILE JSON CUỐI CÙNG ---
# meta_test_data = {}
# for user_id in test_bengali_ids + test_hindi_ids:
#     meta_test_data[user_id] = all_files[user_id]

# final_split_data = {'meta-test': meta_test_data}

# with open(OUTPUT_FILE, 'w') as f:
#     json.dump(final_split_data, f, indent=4)

# print("\n--- HOÀN TẤT TÁI CẤU TRÚC ---")
# print(f"Đã tạo file split cho {len(test_bengali_ids)} người dùng Bengali và {len(test_hindi_ids)} người dùng Hindi.")
# print(f"File được lưu tại: {OUTPUT_FILE}")

In [1]:
# --- CÀI ĐẶT VÀ IMPORTS ---
!rm -rf Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update
!git clone https://github.com/trongjhuongwr/Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update.git
%cd Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update

Cloning into 'Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 44), reused 84 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 188.78 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/kaggle/working/Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update


In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import os
from tqdm.notebook import tqdm
import json
from collections import defaultdict
import re

# Imports từ modules của bạn
from dataloader.meta_dataloader import SignatureEpisodeDataset
from models.feature_extractor import ResNetFeatureExtractor
from models.meta_learner import MetricGenerator
from utils.model_evaluation import evaluate_meta_model

In [3]:
# --- CẤU HÌNH VÀ THỰC THI ĐÁNH GIÁ ---

# Cấu hình
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
K_SHOT = 10 # Số lượng mẫu support để mô hình "học" người dùng mới

# Đường dẫn
MODEL_DIR = '/kaggle/input/best-cedar-model-weights/best_cedar_model' # Sửa lại cho đúng tên dataset model của bạn
BHSIG_SPLIT_FILE = '/kaggle/input/bhsig260-restructured-split/bhsig_restructured_split.json'

# 1. Tải model đã huấn luyện trên CEDAR
feature_extractor = ResNetFeatureExtractor(backbone_name='resnet34', output_dim=512)
metric_generator = MetricGenerator(embedding_dim=512)

fe_path = os.path.join(MODEL_DIR, 'best_feature_extractor.pth')
mg_path = os.path.join(MODEL_DIR, 'best_metric_generator.pth')

feature_extractor.load_state_dict(torch.load(fe_path))
metric_generator.load_state_dict(torch.load(mg_path))

feature_extractor.to(DEVICE)
metric_generator.to(DEVICE)
print("--- Đã tải thành công model huấn luyện trên CEDAR! ---")

# 2. Tạo dataset đánh giá trên BHSig-260
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Vì dataloader của bạn đọc trực tiếp từ file split, chúng ta không cần base_data_dir
bhsig_test_dataset = SignatureEpisodeDataset(
    split_file_path=BHSIG_SPLIT_FILE, 
    base_data_dir=None,  # Đặt là None vì chúng ta dùng đường dẫn tuyệt đối
    split_name='meta-test', 
    k_shot=K_SHOT, 
    n_query_genuine=15, 
    n_query_forgery=15, 
    augment=False,       # <-- Tắt augmentation khi đánh giá
    use_full_path=True   # <-- Báo cho dataloader dùng đường dẫn tuyệt đối
)

# 3. Thực hiện đánh giá
print("\n--- Bắt đầu đánh giá Cross-Dataset trên BHSig-260 ---")
# Chúng ta cần đảm bảo hàm evaluate_meta_model cũng được cập nhật
# Giả sử hàm evaluate_meta_model đã được cập nhật như trong các thảo luận trước
cross_dataset_accuracy = evaluate_meta_model(
    feature_extractor, 
    metric_generator, 
    bhsig_test_dataset, 
    DEVICE
)

print(f"\n\n--- KẾT QUẢ CUỐI CÙNG ---")
print(f"Độ chính xác của model (train trên CEDAR) trên tập test BHSig-260: {cross_dataset_accuracy * 100:.2f}%")
print("----------------------------")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 194MB/s] 


--- Đã tải thành công model huấn luyện trên CEDAR! ---

--- Bắt đầu đánh giá Cross-Dataset trên BHSig-260 ---




--- KẾT QUẢ CUỐI CÙNG ---
Độ chính xác của model (train trên CEDAR) trên tập test BHSig-260: 82.04%
----------------------------
